#Setup & path configuration 


In [4]:
import os, numpy as np, pandas as pd, librosa
from tqdm import tqdm
from python_speech_features import mfcc
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "Notebooks" else Path.cwd()
AUDIO_ROOT   = PROJECT_ROOT / "0_dB_fan" / "fan" / "id_00"  # adjust if needed
FEAT_DIR     = PROJECT_ROOT / "features"
PAD_DIR      = PROJECT_ROOT / "features_padded"

SAMPLE_RATE = 16000
CHUNK_DURATION = 1.0
N_MFCC, N_FILT, N_FFT = 13, 26, 1024

FEAT_DIR.mkdir(exist_ok=True, parents=True)
PAD_DIR.mkdir(exist_ok=True, parents=True)



#Load Dataset & labels 

In [5]:
files, labels = [], []
for lab in ["normal", "abnormal"]:
    folder = AUDIO_ROOT / lab
    if not folder.exists():
        print(f"Warning: {folder} not found")
        continue
    for f in folder.glob("*.wav"):
        files.append(str(f))
        labels.append(0 if lab == "normal" else 1)

df = pd.DataFrame({"filename": files, "label": labels})
len(df), df.head()


(1418,
                                             filename  label
 0  c:\Users\hichr\OneDrive\Desktop\Internship\0_d...      0
 1  c:\Users\hichr\OneDrive\Desktop\Internship\0_d...      0
 2  c:\Users\hichr\OneDrive\Desktop\Internship\0_d...      0
 3  c:\Users\hichr\OneDrive\Desktop\Internship\0_d...      0
 4  c:\Users\hichr\OneDrive\Desktop\Internship\0_d...      0)

MFCC Extraction (with Caching ) (loads existing mfcc_features.npy and mfcc_labels.npy or extracts MFCCs and saves them)

In [ ]:
# Cached MFCC extraction (reuses df, FEAT_DIR, SAMPLE_RATE, CHUNK_DURATION, N_MFCC, N_FILT, N_FFT)

import json, hashlib, os
from pathlib import Path
import numpy as np
import librosa
from tqdm import tqdm
from python_speech_features import mfcc

assert 'df' in globals(), "df not found. Run the file listing cell first."

PARAMS = {
    "sample_rate": SAMPLE_RATE,
    "chunk_sec": CHUNK_DURATION,
    "n_mfcc": N_MFCC,
    "n_filt": N_FILT,
    "n_fft": N_FFT,
    "librosa_mono": True,
    "version": 1,  # bump to force refresh
}

FEAT_DIR.mkdir(exist_ok=True, parents=True)

def file_sig(p: str):
    st = os.stat(p)
    return (p, int(st.st_size), int(st.st_mtime))

manifest = [file_sig(p) for p in df["filename"].tolist()]
manifest_sorted = sorted(manifest)

cache_key_src = json.dumps({"params": PARAMS, "manifest": manifest_sorted}, sort_keys=True).encode()
cache_key = hashlib.md5(cache_key_src).hexdigest()[:16]

cache_dir = FEAT_DIR / f"mfcc_cache_{cache_key}"
features_path = cache_dir / "mfcc_features.npy"
labels_path   = cache_dir / "mfcc_labels.npy"
meta_path     = cache_dir / "meta.json"

if features_path.exists() and labels_path.exists():
    print(f"✅ Using cached MFCCs: {cache_dir.name}")
    mfcc_features = np.load(features_path, allow_pickle=True)
    mfcc_labels   = np.load(labels_path)
else:
    print("🔍 Extracting MFCCs once for this dataset + params…")
    cache_dir.mkdir(parents=True, exist_ok=True)

    sr = PARAMS["sample_rate"]
    chunk_samples = int(PARAMS["chunk_sec"] * sr)
    N_M, N_FILT_, N_FFT_ = PARAMS["n_mfcc"], PARAMS["n_filt"], PARAMS["n_fft"]

    mfcc_features, mfcc_labels = [], []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        x, _ = librosa.load(row["filename"], sr=sr, mono=PARAMS["librosa_mono"])
        n_chunks = len(x) // chunk_samples
        for c in range(n_chunks):   
            chunk = x[c*chunk_samples:(c+1)*chunk_samples]
            try:
                feat = mfcc(chunk, samplerate=sr, numcep=N_M, nfilt=N_FILT_, nfft=N_FFT_)
                mfcc_features.append(feat)            # (frames, n_mfcc)
                mfcc_labels.append(int(row["label"])) # 0/1
            except Exception as e:
                print("Error:", row["filename"], c, e)

    mfcc_features = np.array(mfcc_features, dtype=object)
    mfcc_labels   = np.array(mfcc_labels, dtype=np.int64)

    np.save(features_path, mfcc_features)
    np.save(labels_path,   mfcc_labels)
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump({"params": PARAMS, "manifest_len": len(manifest_sorted)}, f, indent=2)
    print(f"💾 Saved to {cache_dir.name}")

print("features:", len(mfcc_features), "labels:", len(mfcc_labels))


🔍 Extracting MFCCs once for this dataset + params…


100%|██████████| 1418/1418 [01:00<00:00, 23.45it/s]


💾 Saved to mfcc_cache_65694887bc5afe42
features: 14180 labels: 14180


MFCC Padding / Trimming (loads raw MFCCs, pads/truncates them to equal length, and saves to features_padded/)

In [7]:
# Pad/trim to uniform length and cache result (reuses PAD_DIR, mfcc_features, mfcc_labels)

import json, hashlib
import numpy as np

PAD_DIR.mkdir(exist_ok=True, parents=True)

pad_meta = {
    "source_len": int(len(mfcc_features)),
    "strategy": "pad_or_trim_to_max",
}
pad_key = hashlib.md5(json.dumps(pad_meta, sort_keys=True).encode()).hexdigest()[:16]
pad_dir = PAD_DIR / f"padded_{pad_key}"
pad_feat_path = pad_dir / "mfcc_features_padded.npy"
pad_lab_path  = pad_dir / "mfcc_labels.npy"

if pad_feat_path.exists() and pad_lab_path.exists():
    print(f"✅ Using cached padded features: {pad_dir.name}")
    X = np.load(pad_feat_path, allow_pickle=False)
    y = np.load(pad_lab_path)
else:
    pad_dir.mkdir(parents=True, exist_ok=True)
    max_len = max(feat.shape[0] for feat in mfcc_features)
    padded = []
    for feat in mfcc_features:
        if feat.shape[0] < max_len:
            pad = np.pad(feat, ((0, max_len - feat.shape[0]), (0, 0)), mode="constant")
        else:
            pad = feat[:max_len]
        padded.append(pad)
    X = np.array(padded)
    y = mfcc_labels.astype(np.int64)
    np.save(pad_feat_path, X)
    np.save(pad_lab_path,  y)
    print(f"💾 Saved padded to {pad_dir.name} (max_len={max_len})")

print("X shape:", X.shape, "y shape:", y.shape)


💾 Saved padded to padded_6b24d77e5d8b78d7 (max_len=99)
X shape: (14180, 99, 13) y shape: (14180,)


Train / test split + dtype prep (1D CNN ready)

In [9]:
# Cell 4+5 — Train/Test split + dtype prep (1D CNN ready)

from sklearn.model_selection import train_test_split
import numpy as np

# 1) Split (keep class balance with stratify)
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,
    stratify=y
)

# 2) Types / formats for Keras 1D CNN
X_train = X_train.astype(np.float32)   # features as float32
X_test  = X_test.astype(np.float32)
y_train = y_train.astype(np.int32)     # labels as ints (0/1)
y_test  = y_test.astype(np.int32)

# 3) Quick sanity checks
print("X_train shape:", X_train.shape)  # (samples, timesteps, n_mfcc)
print("X_test  shape:", X_test.shape)
print("y_train shape:", y_train.shape, "unique:", np.unique(y_train, return_counts=True))
print("y_test  shape:", y_test.shape,   "unique:", np.unique(y_test, return_counts=True))

# Optional: assert expected rank (3D tensors for 1D CNN)
assert X_train.ndim == 3 and X_test.ndim == 3, "Expected X to be (samples, timesteps, features)"


X_train shape: (11344, 99, 13)
X_test  shape: (2836, 99, 13)
y_train shape: (11344,) unique: (array([0, 1], dtype=int32), array([8088, 3256]))
y_test  shape: (2836,) unique: (array([0, 1], dtype=int32), array([2022,  814]))


Build the 1D -CNN +Callbacks 

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from pathlib import Path

input_shape = (X_train.shape[1], X_train.shape[2])

model = Sequential([
    Conv1D(64, 5, activation="relu", padding="same", input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")  # binary output
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# Handle class imbalance (your data is ~71%/29%)
classes = np.array([0, 1])
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = {int(c): w for c, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# Callbacks
MODEL_DIR = Path(PROJECT_ROOT) / "model"
MODEL_DIR.mkdir(exist_ok=True, parents=True)

ckpt_path = MODEL_DIR / "best_mfcc_1dcnn.keras"
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath=str(ckpt_path), monitor="val_loss", save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)
]


**Purpose:** List each model layer and check if it has trainable weights.  
**Proves:** Confirms which layers learn parameters (Conv1D, Dense, BatchNorm) and which do not (Pooling, Dropout, Flatten).


In [ ]:
for layer in model.layers:
    print(layer.name, layer.trainable, hasattr(layer, 'weights') and layer.weights)


RUN1 

In [14]:
# Cell 7 — Train the model with resume capability and save history
import json
import numpy as np
from tensorflow.keras.models import load_model

# Resume from last saved checkpoint if it exists
if ckpt_path.exists():
    print(f"🔄 Resuming training from checkpoint: {ckpt_path}")
    model = load_model(ckpt_path)
else:
    print("🚀 Starting fresh training")

print("Train shape:", X_train.shape, " Test shape:", X_test.shape)
print("Train class balance:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Using class weights:", class_weight_dict)

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=callbacks,  # Includes EarlyStopping + Checkpoint + LR Scheduler
    shuffle=True,
    verbose=1
)

# Save history after each run
hist_path = MODEL_DIR / "train_history.json"
if hist_path.exists():
    # Merge with previous if resuming
    with open(hist_path, "r", encoding="utf-8") as f:
        old_hist = json.load(f)
    for k, v in history.history.items():
        old_hist[k] = old_hist.get(k, []) + v
    with open(hist_path, "w", encoding="utf-8") as f:
        json.dump(old_hist, f, indent=2)
else:
    with open(hist_path, "w", encoding="utf-8") as f:
        json.dump(history.history, f, indent=2)

print(f"💾 Saved training history to: {hist_path}")


🚀 Starting fresh training
Train shape: (11344, 99, 13)  Test shape: (2836, 99, 13)
Train class balance: {np.int32(0): np.int64(8088), np.int32(1): np.int64(3256)}
Using class weights: {0: np.float64(0.7012858555885262), 1: np.float64(1.742014742014742)}
Epoch 1/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.6868 - loss: 0.6319 - val_accuracy: 0.8036 - val_loss: 0.4338 - learning_rate: 0.0010
Epoch 2/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7642 - loss: 0.4912 - val_accuracy: 0.7987 - val_loss: 0.4583 - learning_rate: 0.0010
Epoch 3/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.7780 - loss: 0.4618 - val_accuracy: 0.7846 - val_loss: 0.4580 - learning_rate: 0.0010
Epoch 4/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8027 - loss: 0.4197 - val_accuracy: 0.7563 - val_loss: 0.5912 - learning_rate: 5.0000e-04
Epoch 5/30
178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.8221 - loss: 0.4011 - val_accuracy: 0.7680 - val_loss: 0.5031 -

TRAIN/VAL/TEST SPLIT 

In [15]:
from sklearn.model_selection import train_test_split

# First split: train+val and test
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)

# Second split: train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.20, random_state=42, stratify=y_trainval
)

# Convert to proper dtypes
X_train = X_train.astype(np.float32)
X_val   = X_val.astype(np.float32)
X_test  = X_test.astype(np.float32)
y_train = y_train.astype(np.int32)
y_val   = y_val.astype(np.int32)
y_test  = y_test.astype(np.int32)

# Sanity check
print(f"Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}")
print("Train labels:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Val labels:", dict(zip(*np.unique(y_val, return_counts=True))))
print("Test labels:", dict(zip(*np.unique(y_test, return_counts=True))))


Train: (9075, 99, 13), Val: (2269, 99, 13), Test: (2836, 99, 13)
Train labels: {np.int32(0): np.int64(6470), np.int32(1): np.int64(2605)}
Val labels: {np.int32(0): np.int64(1618), np.int32(1): np.int64(651)}
Test labels: {np.int32(0): np.int64(2022), np.int32(1): np.int64(814)}


### 📊 Data Split Summary

**Split ratios**  
- **Train**: 9075 samples (~64%)  
- **Validation**: 2269 samples (~16%)  
- **Test**: 2836 samples (~20%)  
✅ Classic 64/16/20 split → enough data for training and unbiased evaluation.

**Class balance in each split**  
- **Train**: 71% normal (6470), 29% abnormal (2605)  
- **Validation**: 71% normal (1618), 29% abnormal (651)  
- **Test**: 71% normal (2022), 29% abnormal (814)  
✅ Class distribution is consistent across all sets → prevents bias in evaluation.




fresh model Build before Run 2


In [17]:
# 🆕 Run 2 — Fresh Model Build
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from pathlib import Path

# 1️⃣ Clear any previous model/graph to avoid conflicts
tf.keras.backend.clear_session()

# 2️⃣ Define input shape based on your NEW split
input_shape = (X_train.shape[1], X_train.shape[2])

# 3️⃣ Build the model (same architecture as before)
model = Sequential([
    Conv1D(64, 5, activation="relu", padding="same", input_shape=input_shape),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")  # binary classification output
])

# 4️⃣ Compile model (fresh optimizer state, set LR explicitly)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# 5️⃣ Show summary for verification
model.summary()

# 6️⃣ Handle class imbalance (based on y_train from new split)
classes = np.array([0, 1])
class_weights = compute_class_weight(class_weight="balanced", classes=classes, y=y_train)
class_weight_dict = {int(c): float(w) for c, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 7️⃣ Callbacks (NEW filename so Run 1 stays safe)
MODEL_DIR = Path(PROJECT_ROOT) / "model"
MODEL_DIR.mkdir(exist_ok=True, parents=True)

ckpt_path = MODEL_DIR / "best_mfcc_1dcnn_run2.keras"
callbacks = [
    EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True),
    ModelCheckpoint(filepath=str(ckpt_path), monitor="val_loss", save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)
]


c:\Users\hichr\OneDrive\Desktop\Internship\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 99, 64)         │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 99, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 49, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 49, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 24, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,353 (1.05 MB)

 Trainable params: 275,713 (1.05 MB)

 Non-trainable params: 640 (2.50 KB)

Class weights: {0: 0.7013137557959814, 1: 1.7418426103646834}


Run 2

In [18]:
# Cell 7 — Train the model (new split, with callbacks & class weights)

import json
import numpy as np

# 🔍 Sanity check
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)
print("Train labels:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Val labels:", dict(zip(*np.unique(y_val, return_counts=True))))
print("Using class weights:", class_weight_dict)

# 🏋️ Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,                 # tune based on results (20–50 common)
    batch_size=64,             # tune for speed/memory
    class_weight=class_weight_dict,
    callbacks=callbacks,       # from Cell 6
    shuffle=True,
    verbose=1
)

# 💾 Save training history for later analysis
hist_path = MODEL_DIR / "train_history_new_split.json"
with open(hist_path, "w", encoding="utf-8") as f:
    json.dump(history.history, f, indent=2)
print(f"Saved training history to: {hist_path}")


Train: (9075, 99, 13) Val: (2269, 99, 13) Test: (2836, 99, 13)
Train labels: {np.int32(0): np.int64(6470), np.int32(1): np.int64(2605)}
Val labels: {np.int32(0): np.int64(1618), np.int32(1): np.int64(651)}
Using class weights: {0: 0.7013137557959814, 1: 1.7418426103646834}
Epoch 1/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 14ms/step - accuracy: 0.6710 - loss: 0.6709 - val_accuracy: 0.7885 - val_loss: 0.4749 - learning_rate: 0.0010
Epoch 2/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7502 - loss: 0.5106 - val_accuracy: 0.7880 - val_loss: 0.4335 - learning_rate: 0.0010
Epoch 3/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7648 - loss: 0.4834 - val_accuracy: 0.7135 - val_loss: 0.5218 - learning_rate: 0.0010
Epoch 4/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.7933 - loss: 0.4502 - val_accuracy: 0.7783 - val_loss: 0.4477 - learning_rate: 0.0010
Epoch 5/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.7985 - loss: 0.4194 - val_accuracy: 0.8202 - va

In [20]:
# 🔄 Reset the model completely
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, Flatten, Dense, BatchNormalization

tf.keras.backend.clear_session()  # clear previous model/optimizer from memory

# Define model with your updated hyperparameters
input_shape = (X_train.shape[1], X_train.shape[2])

model = Sequential([
    Conv1D(64, 5, activation="relu", padding="same", input_shape=input_shape),  # change params here if needed
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),  # change params here
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Conv1D(128, 3, activation="relu", padding="same"),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation="relu"),  # adjust units if testing
    Dropout(0.4),
    Dense(1, activation="sigmoid")  # binary output
])

# Compile with your chosen optimizer & LR
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),  # change LR here if testing
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.summary()


c:\Users\hichr\OneDrive\Desktop\Internship\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 99, 64)         │         4,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 99, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_1 (Conv1D)               │ (None, 49, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 49, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_2 (Conv1D)               │ (None, 24, 128)        │        49,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 24, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 12, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1536)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 276,353 (1.05 MB)

 Trainable params: 275,713 (1.05 MB)

 Non-trainable params: 640 (2.50 KB)

In [21]:
# Cell 7 — Train the model (new split, with callbacks & class weights)

import json
import numpy as np

# 🔍 Sanity check
print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)
print("Train labels:", dict(zip(*np.unique(y_train, return_counts=True))))
print("Val labels:", dict(zip(*np.unique(y_val, return_counts=True))))
print("Using class weights:", class_weight_dict)

# 🏋️ Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=30,                 # tune based on results (20–50 common)
    batch_size=64,             # tune for speed/memory
    class_weight=class_weight_dict,
    callbacks=callbacks,       # from Cell 6
    shuffle=True,
    verbose=1
)

# 💾 Save training history for later analysis
hist_path = MODEL_DIR / "train_history_new_split.json"
with open(hist_path, "w", encoding="utf-8") as f:
    json.dump(history.history, f, indent=2)
print(f"Saved training history to: {hist_path}")


Train: (9075, 99, 13) Val: (2269, 99, 13) Test: (2836, 99, 13)
Train labels: {np.int32(0): np.int64(6470), np.int32(1): np.int64(2605)}
Val labels: {np.int32(0): np.int64(1618), np.int32(1): np.int64(651)}
Using class weights: {0: 0.7013137557959814, 1: 1.7418426103646834}
Epoch 1/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.6657 - loss: 0.6761 - val_accuracy: 0.7104 - val_loss: 0.5289 - learning_rate: 0.0010
Epoch 2/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.7479 - loss: 0.5054 - val_accuracy: 0.7699 - val_loss: 0.4608 - learning_rate: 0.0010
Epoch 3/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.7576 - loss: 0.4710 - val_accuracy: 0.7664 - val_loss: 0.4874 - learning_rate: 5.0000e-04
Epoch 4/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.7748 - loss: 0.4533 - val_accuracy: 0.7669 - val_loss: 0.4591 - learning_rate: 5.0000e-04
Epoch 5/30
142/142 ━━━━━━━━━━━━━━━━━━━━ 3s 22ms/step - accuracy: 0.7905 - loss: 0.4308 - val_accuracy: 0.7